# Using Current Data To Make Recommendations
### Model #1: (Classification - Fully Paid vs. Default) 
### Model #2: ( Regression - Annualized Return %)

In [1]:
import pandas as pd
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
from functions_and_objects import (impute_means_zeros_maxs_X,parse_percentage,
                                   clean_new_LC_data_classification_current,clean_LC_data_classification_eval,
                                   preprocessing_current, preprocessing_future_test, columns_list, 
                                   nan_max_cols, nan_mean_cols, nan_zero_cols, dtype,
                                   one_hot_encode_current, concat_X_and_6ohe_dfs,
                                   prep_all_df_for_classification, view_columns_by_number_of_rows_that_have_nan)
from functions_and_objects import (prep_df_for_regression_current,
                                   calc_annu_return,impute_annu_return_to_y,
                                   scale_current)

import joblib
from sklearn.metrics import (accuracy_score, precision_score, recall_score, #classification
                             f1_score, roc_auc_score, roc_curve,precision_recall_fscore_support, #classification
                             r2_score,mean_squared_error) #regression

%load_ext autoreload
%autoreload 2

### Import New Data from 2018-2019

- These files contain complete loan data for all loans issued through the time period stated, including the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. The file containing loan data through the "present" contains complete loan data for all loans issued through the previous completed calendar quarter.

In [2]:
raw_q12019 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2019Q1.csv',header=1)
raw_q42018 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2018Q4.csv',header=1)
raw_q32018 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2018Q3.csv',header=1)
raw_q22018 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2018Q2.csv',header=1)
raw_q12018 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2018Q1.csv',header=1)
dfs_list = [raw_q42018,
            raw_q32018,
            raw_q22018,
            raw_q12018]

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,129,130,131,134,135,136,139,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Clean Data

In [6]:
#use cleaning function "current" because 
#we're making predictions on current loans
#rather than using labelled/completed data

clean_lc_df_current = clean_new_LC_data_classification_current(dfs_list) #takes in dfs_list, outputs clean_lc_df


/home/jupyter/peervest/functions_and_objects.py:226: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  raw_lc_df['earliest_cr_line'] = pd.to_timedelta(pd.to_datetime(raw_lc_df['earliest_cr_line'])).dt.days


In [9]:
clean_lc_df_future = clean_LC_data_classification_eval(dfs_list)
X_future, y_future = preprocessing_future_test(clean_lc_df_future)
(ohe_home_ownership, ohe_purpose, ohe_zip_code, 
 ohe_application_type, ohe_sub_grade, ohe_emp_title_2) = one_hot_encode_current(X_future)
X_future_classif = concat_X_and_6ohe_dfs(X_future, ohe_home_ownership, ohe_purpose, ohe_zip_code, 
                                         ohe_application_type, ohe_sub_grade, ohe_emp_title_2)

/home/jupyter/peervest/functions_and_objects.py:194: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  raw_lc_df['earliest_cr_line'] = pd.to_timedelta(pd.to_datetime(raw_lc_df['earliest_cr_line'])).dt.days
/home/jupyter/peervest/functions_and_objects.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  y_future['loan_status'] = y_future['loan_status'].astype(int)


In [10]:
#X_future_classif.set_index('index',inplace=True)
prep_all_df_for_classification(X_future_classif)

In [11]:
view_columns_by_number_of_rows_that_have_nan(X_future_classif)

loan_amnt                         67586
funded_amnt                       67586
total_pymnt                       67586
int_rate                          67586
installment                       67586
emp_length                        67586
annual_inc                        67586
dti                               67586
delinq_2yrs                       67586
earliest_cr_line                  67586
fico_range_low                    67586
fico_range_high                   67586
inq_last_6mths                    67586
mths_since_last_delinq            67586
mths_since_last_record            67586
open_acc                          67586
pub_rec                           67586
revol_bal                         67586
revol_util                        67586
total_acc                         67586
out_prncp                         67586
out_prncp_inv                     67586
last_fico_range_high              67586
last_fico_range_low               67586
collections_12_mths_ex_med        67586


In [18]:
view_columns_by_number_of_rows_that_have_nan(X_current_classif)

loan_amnt                         361423
funded_amnt                       361423
total_pymnt                       361423
term                              361423
int_rate                          361423
installment                       361423
emp_length                        361423
annual_inc                        361423
verification_status               361423
addr_state                        361423
dti                               361423
delinq_2yrs                       361423
earliest_cr_line                  361423
fico_range_low                    361423
fico_range_high                   361423
inq_last_6mths                    361423
mths_since_last_delinq            361423
mths_since_last_record            361423
open_acc                          361423
pub_rec                           361423
revol_bal                         361423
revol_util                        361423
total_acc                         361423
out_prncp                         361423
out_prncp_inv   

### Preprocess, no train-test-split: entire dataset is our test data

In [13]:
# takes in clean_lc_df_current, outputs X_current, y_current
X_current, y_current = preprocessing_current(clean_lc_df_current)

In [14]:
print(X_current.shape)
print(y_current.shape)

(361423, 101)
(361423, 1)


### One Hot Encode for both Models

In [15]:
(ohe_home_ownership, ohe_purpose, ohe_zip_code, 
 ohe_application_type, ohe_sub_grade, ohe_emp_title_2) = one_hot_encode_current(X_current)

In [16]:
X_current_classif = concat_X_and_6ohe_dfs(X_current, ohe_home_ownership, ohe_purpose, ohe_zip_code,
                                          ohe_application_type, ohe_sub_grade, ohe_emp_title_2)
X_current_regr = concat_X_and_6ohe_dfs(X_current, ohe_home_ownership, ohe_purpose, ohe_zip_code, 
                                       ohe_application_type, ohe_sub_grade, ohe_emp_title_2)

In [17]:
print(X_current_classif.shape)
print(X_current_regr.shape)

(722846, 1132)
(722846, 1132)


In [19]:
X_current_classif.shape

(722846, 1132)

In [20]:
X_current_classif

,loan_amnt,funded_amnt,total_pymnt,term,int_rate,installment,emp_length,annual_inc,verification_status,addr_state,...,emp_title_2_Technician,emp_title_2_Truck Driver,emp_title_2_Vice President,emp_title_2_driver,emp_title_2_manager,emp_title_2_owner,emp_title_2_sales,emp_title_2_supervisor,emp_title_2_teacher,emp_title_2_truck driver
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Prep X_current for Classification

- reset index

In [ ]:
X_current_classif.set_index('index',inplace=True)

- drop non-numeric & OHE source columns

In [ ]:
prep_all_df_for_classification(X_current_classif) #drops columns inplace

In [ ]:
X_current_classif.shape

### X_current is ready to input into Classification Model

In [ ]:
print(y_current.shape)
y_current.head()

###### Load Model JobLib

In [ ]:
loaded_log_reg_v1 = joblib.load('log_reg_v1.joblib')

In [ ]:
current_class_preds = loaded_log_reg_v1.predict(X_current_classif)
current_class_preds_proba = loaded_log_reg_v1.predict_proba(X_current_classif)

In [ ]:
len(current_class_preds_proba)

In [ ]:
loaded_log_reg_v1.classes_

In [ ]:
current_class_preds_proba[:,0]

In [ ]:
y_current['prob_default'] = current_class_preds_proba[:,0]

###### ------- Scratch Work

In [ ]:
plt.hist(y_current['prob_default'],bins=20,range=(0,1))

In [ ]:
low_prob_default_indices = y_current[y_current['prob_default'] <0.1].index

In [ ]:
low_prob_default_indices

In [ ]:
pd.set_option('display.expand_frame_repr', False)

In [ ]:
y_future['prob_default'] = loaded_log_reg_v1.predict_proba(X_future_classif)[:,0]
low_prob_default_indices_future = y_future[y_future['prob_default'] <0.1].index
low_prob_default_indices_future

In [ ]:
X_current_classif.iloc[low_prob_default_indices,:].head(20).T

In [ ]:
plt.hist(loaded_log_reg_v1.predict_proba(X_future_classif)[:,0],bins=20,range=(0,1))

In [ ]:
loaded_log_reg_v1.predict_proba(X_future_classif)[:,0]

###### ----------

### Prep X_current for Regression

- reset index

In [ ]:
X_current_regr.set_index('index',inplace=True)

- calculate annual return & create new target dataframe

In [ ]:
# returns the annualized return column as a DF (y_future_regr) & both target columns as a dataframe (y_future)
y_current_regr, y_current = impute_annu_return_to_y(X_current_regr,y_current)

- drop non-numeric & OHE source columns

In [ ]:
prep_df_for_regression_current(X_current_regr)

In [ ]:
X_current_regr_scaled = scale_current(X_current_regr)

In [ ]:
X_current_regr_scaled.shape

###### Load Ridge Regression Model JobLib

In [ ]:
loaded_ridge_reg_v1 = joblib.load('ridge_lin_reg_v1.joblib')

In [ ]:
current_return_preds = loaded_ridge_reg_v1.predict(X_current_regr_scaled)

- Positive annu_return values where loan_status==0 likely indicates a successful debt collection process

In [ ]:
current_return_preds

In [ ]:
y_predictions = y_current
y_predictions['return_preds'] = current_return_preds

In [ ]:
y_predictions.drop(columns=['annu_return'],inplace=True)

In [ ]:
y_predictions.describe()

In [ ]:
y_predictions.head()

In [ ]:
X_current_regr.equals(X_current_classif)

In [ ]:
X_current_regr.shape

In [ ]:
y_predictions.shape

In [ ]:
final_df = y_predictions.join(X_current_regr)

In [ ]:
final_df.shape

In [ ]:
final_df.columns.tolist()

# Recommendation Table

# Recommendation Summary Values

# Recommendation Descriptive Visuals